In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/25 12:20:05 WARN Utils: Your hostname, LAPTOP-MMSL4U2D resolves to a loopback address: 127.0.1.1; using 172.18.74.190 instead (on interface eth0)
24/09/25 12:20:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 12:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
trans_final_sdf = spark.read.parquet('../data/merged/merged_transactions_with_tags.parquet')

In [3]:
grouped_sdf = trans_final_sdf.groupBy("merchant_abn",
    F.weekofyear("order_datetime").alias("week"),
    F.year("order_datetime").alias("year"), "merchant_name", "fixed_tags").agg(
    F.sum("dollar_value").alias("revenue"),
    F.sum(F.when(F.col("gender") == "Male", 1).otherwise(0)).alias("male_count"),
    F.sum(F.when(F.col("gender") == "Female", 1).otherwise(0)).alias("female_count"),
    F.sum(F.when(F.col("gender") == "Undisclosed", 1).otherwise(0)).alias("undisclosed_count"),
    F.count("order_id").alias("num_trans"),
    F.count_distinct("postcode").alias("num_postcodes"),
    F.count_distinct("user_id").alias("num_unique_customers"),
    F.avg("consumer_fraud_probability").alias("average_consumer_fraud_probability"),
    F.avg("merchant_fraud_probability").alias("average_merchant_fraud_probability"),
    F.sum(F.when(F.col("is_fraud") == "true", 1).otherwise(0)).alias("fraud_count")
)
grouped_sdf = grouped_sdf.orderBy("merchant_abn", "year", "week")

In [4]:
grouped_sdf=grouped_sdf.withColumn('mean_trans_val', grouped_sdf.revenue/grouped_sdf.num_trans)


In [5]:
grouped_sdf = grouped_sdf\
       .withColumn("description", F.regexp_extract("fixed_tags", r'\[\[([^\]]+)\]', 1)) \
       .withColumn("revenue_band", F.regexp_extract("fixed_tags", r'\[\[[^\]]+\], \[([^\]]+)\]', 1)) \
       .withColumn("take_rate", F.regexp_extract("fixed_tags", r'take rate: (\d+\.?\d*)', 1))

In [6]:
grouped_sdf = grouped_sdf.drop('fixed_tags')
grouped_sdf.show(5, truncate=False)

24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:21:39 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+----+----+-------------+-----------------+----------+------------+-----------------+---------+-------------+--------------------+----------------------------------+----------------------------------+-----------+------------------+-------------------------------------------------------------------------------------+------------+---------+
|merchant_abn|week|year|merchant_name|revenue          |male_count|female_count|undisclosed_count|num_trans|num_postcodes|num_unique_customers|average_consumer_fraud_probability|average_merchant_fraud_probability|fraud_count|mean_trans_val    |description                                                                          |revenue_band|take_rate|
+------------+----+----+-------------+-----------------+----------+------------+-----------------+---------+-------------+--------------------+----------------------------------+----------------------------------+-----------+------------------+------------------------------------------------

In [7]:
grouped_sdf.coalesce(1).write.csv('../data/curated/revenue.csv', header=True)

24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 12:22:35 WARN RowBasedKeyValueBatch: Calling spill() on

In [ ]:
grouped_sdf.printSchema